In [4]:
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import librosa
import numpy as np
import os

In [5]:
SAMPLE_RATE = 22050
SEGMENT_DURATION = 1
N_MELS = 128

In [6]:
def get_logmel_spectrogram(segment):
    mel = librosa.feature.melspectrogram(y=segment, sr=SAMPLE_RATE, n_mels=N_MELS)
    logmel = librosa.power_to_db(mel)
    return logmel

def extract_features(file_path):
    y, _ = librosa.load(file_path, sr=SAMPLE_RATE)
    segment_len = SAMPLE_RATE * SEGMENT_DURATION
    features = []
    for i in range(0, len(y), segment_len):
        segment = y[i:i+segment_len]
        if len(segment) < segment_len:
            break
        logmel = get_logmel_spectrogram(segment)
        features.append(logmel.flatten())
    return features

def create_dataset(folder_path):
    X, y = [], []
    for label, subfolder in enumerate(['background', 'birds']):
        sub_path = os.path.join(folder_path, subfolder)
        for file in os.listdir(sub_path):
            file_path = os.path.join(sub_path, file)
            if file_path.endswith(('.wav', '.mp3')):
                try:
                    segments = extract_features(file_path)
                    X.extend(segments)
                    y.extend([label] * len(segments))
                except:
                    continue
    return np.array(X), np.array(y)

In [7]:
def train_and_evaluate(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LGBMClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    return acc, prec, rec, f1, model

In [8]:
Example usage:
dataset_path = "path_to_your_dataset_folder"
X, y = create_dataset(dataset_path)
acc, prec, rec, f1, model = train_and_evaluate(X, y)

SyntaxError: invalid syntax (250017833.py, line 1)